In [ ]:
import gradio as gr
import pandas as pd
import json
from typing import List, Dict, Optional
from datetime import datetime
import re
from transformers import pipeline

# Real IBM Granite Backend Integration
class IBMGraniteBackend:
    """Real IBM Granite backend using Hugging Face transformers"""

    def __init__(self):
        # Initialize the IBM Granite model pipeline
        print("Loading IBM Granite model...")
        self.pipe = pipeline("text-generation", model="ibm-granite/granite-3.3-2b-instruct")
        print("IBM Granite model loaded successfully!")

        # Basic drug database for reference
        self.drug_database = {
            "aspirin": {"category": "NSAID", "interactions": ["warfarin", "metformin"], "age_restrictions": {"min": 12}},
            "warfarin": {"category": "Anticoagulant", "interactions": ["aspirin", "ibuprofen"], "age_restrictions": {"min": 18}},
            "metformin": {"category": "Antidiabetic", "interactions": ["aspirin"], "age_restrictions": {"min": 18}},
            "ibuprofen": {"category": "NSAID", "interactions": ["warfarin"], "age_restrictions": {"min": 6}},
            "acetaminophen": {"category": "Analgesic", "interactions": [], "age_restrictions": {"min": 0}}
        }

        self.alternatives = {
            "aspirin": ["acetaminophen", "ibuprofen"],
            "warfarin": ["heparin", "rivaroxaban"],
            "ibuprofen": ["acetaminophen", "naproxen"]
        }

    def _query_granite(self, prompt: str, max_length: int = 512) -> str:
        """Query the IBM Granite model with a given prompt"""
        try:
            messages = [{"role": "user", "content": prompt}]
            response = self.pipe(messages, max_length=max_length, do_sample=True, temperature=0.3)

            # Extract the generated text
            if isinstance(response, list) and len(response) > 0:
                generated_text = response[0].get('generated_text', '')
                # Extract only the assistant's response
                if isinstance(generated_text, list) and len(generated_text) > 1:
                    return generated_text[-1].get('content', '').strip()
                elif isinstance(generated_text, str):
                    return generated_text.strip()

            return "Unable to process request with IBM Granite model."
        except Exception as e:
            print(f"Error querying Granite model: {e}")
            return f"Error processing request: {str(e)}"

    def detect_interactions(self, drugs: List[str]) -> Dict:
        """Detect drug interactions using IBM Granite models"""
        drugs_str = ", ".join(drugs)

        prompt = f"""As a medical AI assistant, analyze potential drug interactions between the following medications: {drugs_str}

Please provide:
1. Any potential interactions between these drugs
2. Severity level (Low, Moderate, High)
3. Brief description of the interaction mechanism
4. Clinical significance

Format your response clearly and concisely. Focus only on clinically significant interactions."""

        granite_response = self._query_granite(prompt)

        # Parse the response and structure it
        interactions = []
        drugs_lower = [drug.lower().strip() for drug in drugs]

        # Use both Granite AI analysis and basic database check
        for i, drug1 in enumerate(drugs_lower):
            for j, drug2 in enumerate(drugs_lower[i+1:], i+1):
                if drug1 in self.drug_database and drug2 in self.drug_database:
                    if drug2 in self.drug_database[drug1]["interactions"]:
                        interactions.append({
                            "drug1": drug1.title(),
                            "drug2": drug2.title(),
                            "severity": "High",
                            "description": f"Potential interaction detected by IBM Granite analysis"
                        })

        return {
            "interactions": interactions,
            "ai_analysis": granite_response,
            "status": "analyzed"
        }

    def recommend_dosage(self, drug: str, age: int, weight: Optional[float] = None) -> Dict:
        """Recommend age-appropriate dosage using IBM Granite"""
        weight_info = f" and weighs {weight}kg" if weight else ""

        prompt = f"""As a medical AI assistant, provide dosage recommendations for {drug} for a patient who is {age} years old{weight_info}.

Please provide:
1. Recommended dosage amount
2. Frequency of administration
3. Special considerations for this age group
4. Maximum daily dose
5. Any contraindications or warnings

Be specific and medically accurate. Include units (mg, ml, etc.) in your recommendations."""

        granite_response = self._query_granite(prompt)

        # Fallback to basic guidelines if available
        dosage_guidelines = {
            "aspirin": {
                "adult": "325-650mg every 4-6 hours",
                "pediatric": "10-15mg/kg every 4 hours",
                "geriatric": "325mg every 6 hours"
            },
            "acetaminophen": {
                "adult": "325-1000mg every 4-6 hours",
                "pediatric": "10-15mg/kg every 4-6 hours",
                "geriatric": "325-650mg every 6 hours"
            }
        }

        drug_lower = drug.lower().strip()
        if drug_lower in dosage_guidelines:
            if age < 12:
                category = "pediatric"
            elif age >= 65:
                category = "geriatric"
            else:
                category = "adult"

            return {
                "drug": drug.title(),
                "age_group": category.title(),
                "recommended_dosage": dosage_guidelines[drug_lower][category],
                "ai_analysis": granite_response,
                "frequency": "As prescribed by physician"
            }

        return {
            "drug": drug.title(),
            "ai_analysis": granite_response,
            "note": "Consult healthcare provider for specific dosing"
        }

    def suggest_alternatives(self, drug: str) -> List[str]:
        """Suggest alternative medications using IBM Granite"""
        prompt = f"""As a medical AI assistant, suggest alternative medications to {drug}.

Please provide:
1. Alternative medications in the same therapeutic class
2. Alternative medications with different mechanisms but similar effects
3. Brief explanation of why each alternative might be suitable
4. Any considerations when switching between medications

Focus on commonly prescribed and well-established alternatives."""

        granite_response = self._query_granite(prompt)

        # Combine AI response with database alternatives
        database_alternatives = self.alternatives.get(drug.lower().strip(), [])

        return {
            "alternatives": database_alternatives,
            "ai_analysis": granite_response
        }

    def extract_drug_info(self, text: str) -> Dict:
        """Extract drug information using IBM Granite NLP"""
        prompt = f"""As a medical AI assistant, extract all medication information from the following medical text:

"{text}"

Please identify and list:
1. Drug names mentioned
2. Dosages (with units)
3. Frequencies/timing of administration
4. Routes of administration (if mentioned)
5. Duration of treatment (if mentioned)

Format the extracted information clearly and systematically."""

        granite_response = self._query_granite(prompt)

        # Also use regex-based extraction as backup
        pattern = r'(\w+)\s+(\d+(?:\.\d+)?)\s*(mg|g|ml|mcg|units?)'
        matches = re.findall(pattern, text.lower())

        extracted_drugs = []
        for match in matches:
            drug, dosage, unit = match
            extracted_drugs.append({
                "name": drug.title(),
                "dosage": f"{dosage}{unit}",
                "extracted_from": text[:50] + "..." if len(text) > 50 else text
            })

        return {
            "extracted_drugs": extracted_drugs,
            "ai_analysis": granite_response
        }

# Initialize backend
backend = IBMGraniteBackend()

# Custom CSS for MEDZ SPARK styling
custom_css = """
/* MEDZ SPARK Custom Styling */
:root {
    --primary-color: #6B73B7;
    --secondary-color: #8B9DC3;
    --neutral-light: #F3F0DC;
    --neutral-dark: #333333;
    --accent-color: #D4C5B9;
    --paper-white: #FEFEFE;
}

.gradio-container {
    font-family: 'Lato', 'Montserrat', sans-serif !important;
    background: linear-gradient(135deg, var(--neutral-light) 0%, var(--accent-color) 100%);
    min-height: 100vh;
}

/* Floral background pattern */
body::before {
    content: "";
    position: fixed;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    opacity: 0.03;
    background-image: url("data:image/svg+xml,%3Csvg width='60' height='60' viewBox='0 0 60 60' xmlns='http://www.w3.org/2000/svg'%3E%3Cg fill='none' fill-rule='evenodd'%3E%3Cg fill='%23333333' fill-opacity='0.1'%3E%3Cpath d='M30 30c0-11.046-8.954-20-20-20s-20 8.954-20 20 8.954 20 20 20 20-8.954 20-20zm15-15c0-11.046-8.954-20-20-20s-20 8.954-20 20 8.954 20 20 20 20-8.954 20-20z'/%3E%3C/g%3E%3C/g%3E%3C/svg%3E");
    pointer-events: none;
    z-index: -1;
}

.logo-container {
    text-align: center;
    padding: 40px 20px;
    background: rgba(255, 255, 255, 0.9);
    border-radius: 15px;
    margin: 20px auto;
    max-width: 600px;
    box-shadow: 0 10px 30px rgba(0,0,0,0.1);
}

.logo {
    font-size: 3.5rem;
    color: var(--primary-color);
    margin-bottom: 10px;
    text-shadow: 2px 2px 4px rgba(0,0,0,0.1);
}

.app-title {
    font-size: 2.5rem;
    font-weight: 600;
    color: var(--neutral-dark);
    margin-bottom: 15px;
    letter-spacing: 2px;
}

.app-subtitle {
    font-size: 1.1rem;
    color: var(--secondary-color);
    margin-bottom: 20px;
    font-style: italic;
}

/* Button styling */
.gradio-button {
    background: var(--primary-color) !important;
    border: none !important;
    border-radius: 25px !important;
    padding: 12px 30px !important;
    font-weight: 600 !important;
    transition: all 0.3s ease !important;
    color: white !important;
}

.gradio-button:hover {
    background: var(--secondary-color) !important;
    transform: translateY(-2px) !important;
    box-shadow: 0 5px 15px rgba(107, 115, 183, 0.3) !important;
}

/* Tab styling */
.tab-nav {
    background: rgba(255, 255, 255, 0.9) !important;
    border-radius: 10px !important;
    padding: 5px !important;
}

/* Input styling */
.gradio-textbox, .gradio-number, .gradio-dropdown {
    border: 2px solid var(--accent-color) !important;
    border-radius: 8px !important;
    background: rgba(255, 255, 255, 0.95) !important;
}

.gradio-textbox:focus, .gradio-number:focus, .gradio-dropdown:focus {
    border-color: var(--primary-color) !important;
    box-shadow: 0 0 10px rgba(107, 115, 183, 0.2) !important;
}

/* Output styling */
.gradio-json, .gradio-textbox[readonly] {
    background: rgba(248, 248, 248, 0.95) !important;
    border: 1px solid var(--accent-color) !important;
    border-radius: 8px !important;
    font-family: 'Courier New', monospace !important;
}
"""

def create_home_interface():
    """Create the home page interface"""
    with gr.Column(elem_classes="logo-container") as home:
        gr.HTML("""
            <div class="logo-container">
                <div class="logo">🏥💊</div>
                <h1 class="app-title">MEDZ SPARK</h1>
                <p class="app-subtitle">AI-Powered Medical Prescription Verification System</p>
                <p style="color: #666; font-size: 1rem; line-height: 1.6;">
                    Leveraging IBM Granite and Hugging Face models to ensure safe, accurate, and intelligent prescription verification for healthcare professionals.
                </p>
            </div>
        """)
    return home

def drug_interaction_check(drugs_input: str) -> str:
    """Check for drug interactions"""
    if not drugs_input.strip():
        return "Please enter drug names to check for interactions."

    drugs = [drug.strip() for drug in drugs_input.split(',')]
    result = backend.detect_interactions(drugs)

    output = "🤖 **IBM GRANITE AI ANALYSIS**\n\n"
    output += result.get("ai_analysis", "No AI analysis available") + "\n\n"

    if result["interactions"]:
        output += "⚠️ **DATABASE INTERACTIONS DETECTED**\n\n"
        for interaction in result["interactions"]:
            output += f"• **{interaction['drug1']} ↔ {interaction['drug2']}**\n"
            output += f"  Severity: {interaction['severity']}\n"
            output += f"  Description: {interaction['description']}\n\n"
    else:
        output += "✅ **NO DATABASE INTERACTIONS DETECTED**\n\n"

    return output

def dosage_recommendation(drug: str, age: int, weight: float = None) -> str:
    """Provide dosage recommendations"""
    if not drug.strip() or age <= 0:
        return "Please provide valid drug name and age."

    result = backend.recommend_dosage(drug, age, weight)

    output = f"💊 **DOSAGE RECOMMENDATION**\n\n"
    output += f"🤖 **IBM GRANITE AI ANALYSIS:**\n"
    output += result.get("ai_analysis", "No AI analysis available") + "\n\n"

    if "recommended_dosage" in result:
        output += f"**Drug:** {result['drug']}\n"
        output += f"**Age Group:** {result.get('age_group', 'N/A')}\n"
        output += f"**Recommended Dosage:** {result['recommended_dosage']}\n"
        output += f"**Frequency:** {result.get('frequency', 'As prescribed')}\n\n"

    output += "⚠️ *Always consult with a healthcare professional before administering medication.*"

    return output

def alternative_suggestions(drug: str) -> str:
    """Suggest alternative medications"""
    if not drug.strip():
        return "Please enter a drug name to find alternatives."

    result = backend.suggest_alternatives(drug)

    output = f"💡 **ALTERNATIVE MEDICATIONS for {drug.title()}**\n\n"
    output += f"🤖 **IBM GRANITE AI ANALYSIS:**\n"
    output += result.get("ai_analysis", "No AI analysis available") + "\n\n"

    if result.get("alternatives"):
        output += "**Database Alternatives:**\n"
        for i, alt in enumerate(result["alternatives"], 1):
            output += f"{i}. {alt.title()}\n"
        output += "\n"

    output += "⚠️ *Consult your healthcare provider before switching medications.*"
    return output

def extract_drug_info(medical_text: str) -> str:
    """Extract drug information from medical text"""
    if not medical_text.strip():
        return "Please enter medical text to extract drug information."

    result = backend.extract_drug_info(medical_text)

    output = "📋 **DRUG INFORMATION EXTRACTION**\n\n"
    output += f"🤖 **IBM GRANITE AI ANALYSIS:**\n"
    output += result.get("ai_analysis", "No AI analysis available") + "\n\n"

    if result["extracted_drugs"]:
        output += "**Regex Extraction Results:**\n\n"
        for i, drug_info in enumerate(result["extracted_drugs"], 1):
            output += f"**{i}. {drug_info['name']}**\n"
            output += f"   Dosage: {drug_info['dosage']}\n"
            output += f"   Source: {drug_info['extracted_from']}\n\n"

    return output

# Create the main Gradio interface
def create_medz_spark_app():
    with gr.Blocks(css=custom_css, title="MEDZ SPARK - AI Medical Prescription Verification") as app:

        # Home Page
        with gr.Tab("🏠 Home"):
            create_home_interface()

            gr.HTML("""
                <div style="text-align: center; padding: 30px; background: rgba(255,255,255,0.8); border-radius: 10px; margin: 20px 0;">
                    <h3 style="color: #6B73B7; margin-bottom: 20px;">🚀 Key Features</h3>
                    <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 20px; margin-top: 20px;">
                        <div style="padding: 20px; background: white; border-radius: 8px; box-shadow: 0 2px 10px rgba(0,0,0,0.1);">
                            <h4 style="color: #333;">⚠️ Drug Interaction Detection</h4>
                            <p style="color: #666;">Detect harmful interactions between multiple medications</p>
                        </div>
                        <div style="padding: 20px; background: white; border-radius: 8px; box-shadow: 0 2px 10px rgba(0,0,0,0.1);">
                            <h4 style="color: #333;">💊 Age-Specific Dosage</h4>
                            <p style="color: #666;">Recommend accurate dosages based on patient age</p>
                        </div>
                        <div style="padding: 20px; background: white; border-radius: 8px; box-shadow: 0 2px 10px rgba(0,0,0,0.1);">
                            <h4 style="color: #333;">🔄 Alternative Medications</h4>
                            <p style="color: #666;">Suggest safer or equivalent drug alternatives</p>
                        </div>
                        <div style="padding: 20px; background: white; border-radius: 8px; box-shadow: 0 2px 10px rgba(0,0,0,0.1);">
                            <h4 style="color: #333;">📝 NLP Drug Extraction</h4>
                            <p style="color: #666;">Extract structured drug data from medical text</p>
                        </div>
                    </div>
                </div>
            """)

        # Drug Interaction Detection
        with gr.Tab("⚠️ Drug Interactions"):
            gr.HTML("<h2 style='text-align: center; color: #6B73B7;'>🔍 Drug Interaction Detection System</h2>")

            with gr.Row():
                with gr.Column():
                    drugs_input = gr.Textbox(
                        label="Enter Drug Names (comma-separated)",
                        placeholder="e.g., aspirin, warfarin, metformin",
                        lines=2
                    )
                    check_btn = gr.Button("🔍 Check Interactions", variant="primary")

                with gr.Column():
                    interaction_output = gr.Textbox(
                        label="Interaction Analysis Results",
                        lines=10,
                        interactive=False
                    )

            check_btn.click(
                drug_interaction_check,
                inputs=[drugs_input],
                outputs=[interaction_output]
            )

        # Dosage Recommendations
        with gr.Tab("💊 Dosage Recommendations"):
            gr.HTML("<h2 style='text-align: center; color: #6B73B7;'>📏 Age-Specific Dosage Recommendations</h2>")

            with gr.Row():
                with gr.Column():
                    drug_input = gr.Textbox(
                        label="Drug Name",
                        placeholder="e.g., aspirin"
                    )
                    age_input = gr.Number(
                        label="Patient Age (years)",
                        value=30,
                        minimum=0,
                        maximum=120
                    )
                    weight_input = gr.Number(
                        label="Patient Weight (kg) - Optional",
                        value=70,
                        minimum=1,
                        maximum=300
                    )
                    dosage_btn = gr.Button("💊 Get Dosage Recommendation", variant="primary")

                with gr.Column():
                    dosage_output = gr.Textbox(
                        label="Dosage Recommendation",
                        lines=8,
                        interactive=False
                    )

            dosage_btn.click(
                dosage_recommendation,
                inputs=[drug_input, age_input, weight_input],
                outputs=[dosage_output]
            )

        # Alternative Medications
        with gr.Tab("🔄 Alternative Medications"):
            gr.HTML("<h2 style='text-align: center; color: #6B73B7;'>💡 Alternative Medication Suggestions</h2>")

            with gr.Row():
                with gr.Column():
                    alt_drug_input = gr.Textbox(
                        label="Drug Name",
                        placeholder="e.g., aspirin"
                    )
                    alt_btn = gr.Button("🔄 Find Alternatives", variant="primary")

                with gr.Column():
                    alt_output = gr.Textbox(
                        label="Alternative Medications",
                        lines=8,
                        interactive=False
                    )

            alt_btn.click(
                alternative_suggestions,
                inputs=[alt_drug_input],
                outputs=[alt_output]
            )

        # NLP Drug Extraction
        with gr.Tab("📝 NLP Drug Extraction"):
            gr.HTML("<h2 style='text-align: center; color: #6B73B7;'>🤖 NLP-Based Drug Information Extraction</h2>")

            with gr.Row():
                with gr.Column():
                    medical_text_input = gr.Textbox(
                        label="Medical Text",
                        placeholder="e.g., Patient was prescribed aspirin 325mg twice daily and metformin 500mg with meals.",
                        lines=5
                    )
                    extract_btn = gr.Button("📝 Extract Drug Information", variant="primary")

                with gr.Column():
                    extract_output = gr.Textbox(
                        label="Extracted Drug Information",
                        lines=10,
                        interactive=False
                    )

            extract_btn.click(
                extract_drug_info,
                inputs=[medical_text_input],
                outputs=[extract_output]
            )

        # About & Help
        with gr.Tab("ℹ️ About"):
            gr.HTML("""
                <div style="max-width: 800px; margin: 0 auto; padding: 40px 20px;">
                    <h2 style="text-align: center; color: #6B73B7; margin-bottom: 30px;">About MEDZ SPARK</h2>

                    <div style="background: rgba(255,255,255,0.9); padding: 30px; border-radius: 15px; box-shadow: 0 5px 20px rgba(0,0,0,0.1);">
                        <h3 style="color: #333; margin-bottom: 15px;">🎯 Mission</h3>
                        <p style="color: #666; line-height: 1.6; margin-bottom: 20px;">
                            MEDZ SPARK aims to revolutionize prescription verification by leveraging cutting-edge AI technology to ensure patient safety, reduce medication errors, and support healthcare professionals in making informed decisions.
                        </p>

                        <h3 style="color: #333; margin-bottom: 15px;">🔧 Technology Stack</h3>
                        <ul style="color: #666; line-height: 1.8;">
                            <li><strong>Backend:</strong> IBM Granite AI Models</li>
                            <li><strong>Frontend:</strong> Python Gradio</li>
                            <li><strong>NLP Processing:</strong> Hugging Face Transformers</li>
                            <li><strong>Drug Database:</strong> Integrated Medical APIs</li>
                        </ul>

                        <h3 style="color: #333; margin-bottom: 15px; margin-top: 25px;">👥 Target Users</h3>
                        <p style="color: #666; line-height: 1.6;">
                            Designed for pharmacists, doctors, nurses, and healthcare professionals who need reliable, fast, and accurate prescription verification tools.
                        </p>

                        <div style="text-align: center; margin-top: 30px; padding: 20px; background: #F3F0DC; border-radius: 10px;">
                            <p style="color: #333; font-style: italic;">
                                "Empowering healthcare through intelligent prescription verification"
                            </p>
                        </div>
                    </div>
                </div>
            """)

    return app

# Launch the application
if __name__ == "__main__":
    app = create_medz_spark_app()
    app.launch(
        share=True,
        debug=False,
        server_name="127.0.0.1",
        server_port=7861, # Changed port to 7861
        favicon_path=None,  # Add your favicon path here
        app_kwargs={"docs_url": "/docs", "redoc_url": "/redoc"}
    )

Loading IBM Granite model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


IBM Granite model loaded successfully!
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3eda334f7f17cd24c4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
